# Ejemplo de un Sistema de Recomendación

Se usará datos adaptados de movielens (https://grouplens.org/datasets/movielens/). Dados nuevos usuarios, se desea predecir su "rating" sobre películas que aún no han visto (IDpelicula), con base en usuarios anteriores.


In [1]:
# Solo necesario si se usa Google Colab
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Ejemplo_SistRecomendacion').getOrCreate()

In [3]:
# Lectura de datos
df = spark.read.csv('/content/ratings_peliculas.csv', inferSchema=True, header=True)
df.show(5)

+----------+------+---------+
|IDpelicula|rating|IDusuario|
+----------+------+---------+
|         2|   3.0|        0|
|         3|   1.0|        0|
|         5|   2.0|        0|
|         9|   4.0|        0|
|        11|   1.0|        0|
+----------+------+---------+
only showing top 5 rows



In [4]:
# Dividir los datos en entrenamiento y prueba (80% - 20%)
df_train, df_test = df.randomSplit([0.8, 0.2])
df_train.show(5)

+----------+------+---------+
|IDpelicula|rating|IDusuario|
+----------+------+---------+
|         0|   1.0|        3|
|         0|   1.0|        5|
|         0|   1.0|        8|
|         0|   1.0|       11|
|         0|   1.0|       13|
+----------+------+---------+
only showing top 5 rows



In [5]:
# Observar la cantidad de usuarios y de películas
df_train.describe().show()

+-------+------------------+-----------------+------------------+
|summary|        IDpelicula|           rating|         IDusuario|
+-------+------------------+-----------------+------------------+
|  count|              1224|             1224|              1224|
|   mean|  50.0187908496732|1.776143790849673|14.444444444444445|
| stddev|28.998725094067876|1.195064598196123|  8.58578100726304|
|    min|                 0|              1.0|                 0|
|    max|                99|              5.0|                29|
+-------+------------------+-----------------+------------------+



### Modelo ALS: Alternating Least Squares matrix factorization

In [6]:
# Construir el modelo usando ALS
from pyspark.ml.recommendation import ALS

als = ALS(userCol="IDusuario", itemCol="IDpelicula", 
          ratingCol="rating", 
          maxIter=5, regParam=0.01)

modelo = als.fit(df_train)

### Predicciones

In [7]:
# Predicciones sobre el conjunto de prueba
predicciones = modelo.transform(df_test)

predicciones.show(10)

+----------+------+---------+----------+
|IDpelicula|rating|IDusuario|prediction|
+----------+------+---------+----------+
|         3|   1.0|        9| 0.5069505|
|         3|   1.0|       28|0.18537688|
|         2|   3.0|        0| 1.6898147|
|         2|   4.0|       10|  2.287214|
|         2|   4.0|       21| 2.7229218|
|         2|   1.0|       26| 1.6648624|
|         2|   4.0|       28| 1.9959936|
|         0|   1.0|        6| 1.6630553|
|         0|   1.0|       15| 1.6701224|
|         0|   1.0|       27| 1.3067544|
+----------+------+---------+----------+
only showing top 10 rows



In [8]:
# Predicción para un solo usuario (usuario 11): se mantiene solo IDusuario y IDpellicula
usuario = df_test.filter(df_test['IDusuario']==11).select(['IDpelicula','IDusuario'])

# Muestra el usuario y las películas que podría ver
usuario.show()

# Recomendaciones de las películas que vería
recomendaciones = modelo.transform(usuario)

# Mostrar resultados ordenados
recomendaciones.orderBy('prediction', ascending=False).show()

+----------+---------+
|IDpelicula|IDusuario|
+----------+---------+
|        10|       11|
|        18|       11|
|        19|       11|
|        20|       11|
|        32|       11|
|        35|       11|
|        37|       11|
|        45|       11|
|        50|       11|
|        66|       11|
|        71|       11|
|        72|       11|
+----------+---------+

+----------+---------+-----------+
|IDpelicula|IDusuario| prediction|
+----------+---------+-----------+
|        18|       11|  5.8179164|
|        19|       11|  3.0142527|
|        20|       11|   2.375272|
|        35|       11|  2.1879346|
|        66|       11|  2.0098839|
|        50|       11|  1.9482667|
|        32|       11|  1.9072865|
|        10|       11|  1.0953431|
|        45|       11|  0.9442185|
|        72|       11|  0.4021974|
|        37|       11| 0.31508586|
|        71|       11|0.081183285|
+----------+---------+-----------+



### Evaluación

Se brinda una medida referencial. Sin embargo, es bastante complicado saber de manera concluyente cuán bien un sistema de recomendación funciona, dado que hay mucha subjetividad del usuario.

In [9]:
from pyspark.ml.evaluation import RegressionEvaluator

# Se usará RMSE (Root Mean Squared Error) para evaluar la predicción vs rating
evaluador = RegressionEvaluator(labelCol="rating", predictionCol="prediction",
                                metricName="rmse")

rmse = evaluador.evaluate(predicciones)
print("Error RMSE: {:.2f}".format(rmse))

Error RMSE: 1.54
